##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Load a prebuilt dataset.
1. Build a neural network machine learning model that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.

## Set up TensorFlow

Import TensorFlow into your program to get started. TensorFlow is a widely used open source library developed by Google and used for machine learning (ML) and artificial intelligence (AI). TensorFlow provides a comprehensive framework for developing and deploying ML models.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)

## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). MNIST is an extensive dataset containing images of handwritten digits (0 to 9). We load two datasets. One for training and one for testing the model.

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data() # x is the image, y is the corresponding label

We can display the content of such an image as an example. The corresponding label should match the image.

In [ ]:
# pick third image (indexing starts at 0 in Python) from all 60.000 train images
example_image = x_train[2]

# plot the 28x28 digit in grayscale with the integrated imshow() function
plt.imshow(example_image, cmap=plt.cm.binary)

# we see the training_label[2] = 4 is correct
print('label:', y_train[2])

The pixel values of the images range from 0 through 255. Scale these values to a range of 0 to 1 by dividing the values by `255.0`. This is also called noramlization and converts the sample data from integers to floating-point numbers:

In [ ]:
print(x_train.dtype) # type of the variable
x_train, x_test = x_train / 255.0, x_test / 255.
print(x_train.dtype)

## Build a machine learning model

Now we want to build a `tf.keras.Sequential` model. A model contains od different layers. Layers are functions with a known mathematical structure that can be reused and have trainable variables. [`Sequential`](https://www.tensorflow.org/guide/keras/sequential_model) is useful for stacking layers where each layer has one input [tensor](https://www.tensorflow.org/guide/tensor) and one output tensor. This model uses the [`Flatten`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten), [`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense), and [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores. For MNIST we get 10 values, one for each class (0-9). Each value indicates how strongly the model thinks that the input example belongs to the class.

In [ ]:
predictions = model(x_train[:1]).numpy()
print(predictions)

Since these values are unscaled we need another function to scale them. The `tf.nn.softmax` function is used to convert these values into *probabilities* (values between 0 and 1) for each class. A probability of 1 would mean, that the model is 100 % sure that the example belongs to this class. 

In [ ]:
probabilitites = tf.nn.softmax(predictions).numpy()
print(probabilitites)

In the next step we want to define a loss function for training. A loss function measures the difference between the predicted values of the model and the actual target values. The primary purpose of a loss function is to guide the training of the model by calculating a value that indicates how well the model already makes its predictions. The model is then trained to minimize the loss. In this example we are using `losses.SparseCategoricalCrossentropy` as a loss function:

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Before you start training, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`. The optimizer is an algorithm that adjusts the internal parameters of the model during training so that the loss is minimal. Set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to start the training. This will adjust your model parameters and minimize the loss.The training takes place in several epochs. Within an epoch, the model is trained once on all training data. Therefore the training samples are divided into minibatches. The batch size is used to determine how many training samples the model should process at once.

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

The bars show how many iterations were carried out within the epoch. An iteration is the process by which the model goes through one minibatch. We can therefore calculate the iterations of an epoch from:
Iterations = Training samples / Batchsize = 60000 / 32 = 1875

After training the model we can now evaluate it. The `Model.evaluate` method checks the model's performance, usually on a [validation set](https://developers.google.com/machine-learning/glossary#validation-set) or [test set](https://developers.google.com/machine-learning/glossary#test-set).

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

## Conclusion

Congratulations! You have trained a machine learning model using a prebuilt dataset using the [Keras](https://www.tensorflow.org/guide/keras/overview) API.

For more examples of using Keras, check out the [tutorials](https://www.tensorflow.org/tutorials/keras/). To learn more about building models with Keras, read the [guides](https://www.tensorflow.org/guide/keras). If you want learn more about loading and preparing data, see the tutorials on [image data loading](https://www.tensorflow.org/tutorials/load_data/images) or [CSV data loading](https://www.tensorflow.org/tutorials/load_data/csv).
